# A Meeting Analysis Use Case

Copyright 2025-2026, Denis Rothman

The notebook's goal is to demonstrate how to guide an AI through a multi-step analytical process, moving from a raw transcript to actionable insights, thereby training both the user and the AI.

This notebook is a step-by-step guide on how you can engineer a structured context . This guide will serve as the architectural blueprint for your code, explaining not just the *what* but the *why* at each stage.

<img src="https://raw.githubusercontent.com/Denis2054/Context-Engineering-for-Multi-Agent-Systems/main/Chapter01/Use_Case_Flowhart.png" alt="Flowchart" width="800"/>


-----

### Setup and Preliminaries

This section handles the basic configuration.

1.  **Cell 1: Install Libraries**

      * Install the necessary OpenAI library.

In [ ]:
# Cell 1: Installation
!pip install tqdm==4.67.1 --upgrade
!pip install openai==2.14.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.12.0
    Uninstalling openai-2.12.0:
      Successfully uninstalled openai-2.12.0


2.  **Cell 2: Imports and API Key**

      * Import the library and securely prompt for the user's API key. This is better than hard-coding it.

In [ ]:
# Cell 2: Imports and API Key Setup
# We will use the OpenAI library to interact with the LLM and Google Colab's
# secret manager to securely access your API key.

import os
from openai import OpenAI
from google.colab import userdata

# Load the API key from Colab secrets, set the env var, then init the client
try:
    api_key = userdata.get("API_KEY")
    if not api_key:
        raise userdata.SecretNotFoundError("API_KEY not found.")

    # Set environment variable for downstream tools/libraries
    os.environ["OPENAI_API_KEY"] = api_key

    # Create client (will read from OPENAI_API_KEY)
    client = OpenAI()
    print("OpenAI API key loaded and environment variable set successfully.")

except userdata.SecretNotFoundError:
    print('Secret "API_KEY" not found.')
    print('Please add your OpenAI API key to the Colab Secrets Manager.')
except Exception as e:
    print(f"An error occurred while loading the API key: {e}")


OpenAI API key loaded and environment variable set successfully.


3.  **Cell 3: The Raw Data (The "Crime Scene")**

      * Define the meeting transcript as a multi-line string. This is our primary data source.

In [ ]:
# Cell 3: The Full Meeting Transcript
meeting_transcript = """
        Tom: Morning all. Coffee is still kicking in.
        Sarah: Morning, Tom. Right, let's jump in. Project Phoenix timeline. Tom, you said the backend components are on track?
        Tom: Mostly. We hit a small snag with the payment gateway integration. It's... more complex than the docs suggested. We might need another three days.
        Maria: Three days? Tom, that's going to push the final testing phase right up against the launch deadline. We don't have that buffer.
        Sarah: I agree with Maria. What's the alternative, Tom?
        Tom: I suppose I could work over the weekend to catch up. I'd rather not, but I can see the bind we're in.
        Sarah: Appreciate that, Tom. Let's tentatively agree on that. Maria, what about the front-end?
        Maria: We're good. In fact, we're a bit ahead. We have some extra bandwidth.
        Sarah: Excellent. Okay, one last thing. The marketing team wants to do a big social media push on launch day. Thoughts?
        Tom: Seems standard.
        Maria: I think that's a mistake. A big push on day one will swamp our servers if there are any initial bugs. We should do a soft launch, invite-only for the first week, and then do the big push. More controlled.
        Sarah: That's a very good point, Maria. A much safer strategy. Let's go with that. Okay, great meeting. I'll send out a summary.
        Tom: Sounds good. Now, more coffee.
        """

-----

### Layer 1: Establishing the scope (The 'What')

Here, we define the scope of the analysis. Each step's output will inform the next, creating a chain of context.

1.  **Cell 4: g2 - Set the Signal-to-Noise Ratio**

      * We'll start by cleaning the data. The prompt explicitly tells the AI to separate substantive content from conversational noise.

In [ ]:
# Cell 4: g2 - Isolating Content from Noise
prompt_g2 = f"""
        Analyze the following meeting transcript. Your task is to isolate the substantive content from the conversational noise.
        - Substantive content includes: decisions made, project updates, problems raised, and strategic suggestions.
        - Noise includes: greetings, pleasantries, and off-topic remarks (like coffee).
        Return ONLY the substantive content.

        Transcript:
        ---
        {meeting_transcript}
        ---
        """

In [ ]:
from openai import OpenAI

try:
    client = OpenAI()

    response_g2 = client.chat.completions.create(
        model="gpt-5.2",
        messages=[
            {"role": "user", "content": prompt_g2}
        ]
    )

    substantive_content = response_g2.choices[0].message.content
    print("--- SUBSTANTIVE CONTENT ---")
    print(substantive_content)

except Exception as e:
    print(f"An error occurred: {e}")

--- SUBSTANTIVE CONTENT ---
- Project Phoenix backend is mostly on track, but payment gateway integration is more complex than expected and would add ~3 days.
- Concern raised: a 3-day slip would compress final testing against the launch deadline due to lack of buffer.
- Tentative decision: Tom will work over the weekend to recover the schedule and avoid delaying the timeline.
- Front-end status: ahead of schedule with extra bandwidth available.
- Marketing/launch strategy issue raised: a large social media push on launch day could overload servers if initial bugs exist.
- Decision: do a soft launch (invite-only) for the first week, then follow with the big social media push.


2.  **Cell 5: g3 - Define the Scope of Time (Simulated RAG)**

      * We'll simulate a RAG context by providing a "previous" summary and asking for what's new. This teaches the user the importance of historical context.

In [ ]:
# Cell 5: g3 - Identifying NEW Information (Simulated RAG)
previous_summary = "In our last meeting, we finalized the goals for Project Phoenix and assigned backend work to Tom and front-end to Maria."

prompt_g3 = f"""
Context: The summary of our last meeting was: "{previous_summary}"

Task: Analyze the following substantive content from our new meeting. Identify and summarize ONLY the new developments, problems, or decisions that have occurred since the last meeting.

New Meeting Content:
---
{substantive_content}
---
"""
from openai import OpenAI
from google.colab import userdata


# Your chat completion request
try:
    response_g3 = client.chat.completions.create(
        model="gpt-5.2",
        messages=[{"role": "user", "content": prompt_g3}]
    )
    new_developments = response_g3.choices[0].message.content
    print("--- NEW DEVELOPMENTS SINCE LAST MEETING ---")
    print(new_developments)
except Exception as e:
    print(f"An error occurred: {e}")


--- NEW DEVELOPMENTS SINCE LAST MEETING ---
- **Backend risk identified:** Payment gateway integration is more complex than expected, potentially adding **~3 days** to the backend schedule (though backend is otherwise mostly on track).
- **Schedule/testing concern raised:** A **3-day slip** would **compress final testing** and threaten the launch timeline due to limited buffer.
- **Tentative mitigation decision:** **Tom will work over the weekend** to try to recover the schedule and avoid delaying the timeline.
- **Front-end update:** Front-end work is **ahead of schedule**, creating **extra bandwidth** availability.
- **Launch strategy problem identified:** A large launch-day social media push could **overload servers** if early bugs exist.
- **Launch strategy decision:** Proceed with a **soft launch (invite-only) for the first week**, then do the **major social media push** afterward.


-----

### Layer 2: Conducting the Investigation (The 'How')

Now we move from identifying facts to generating insights, the core of the semantic context interpreation journey.

1.  **Cell 6: g4 - Identify the Key Threads**

      * This is a crucial step. The prompt asks the AI to read between the lines.

In [ ]:
# Cell 6: g4 - Uncovering Implicit Threads
prompt_g4 = f"""
Task: Analyze the following meeting content for implicit social dynamics and unstated feelings. Go beyond the literal words.
- Did anyone seem hesitant or reluctant despite agreeing to something?
- Were there any underlying disagreements or tensions?
- What was the overall mood?

Meeting Content:
---
{substantive_content}
---
"""
try:
    response_g4 = client.chat.completions.create(
        model="gpt-5.2",
        messages=[{"role": "user", "content": prompt_g4}]
    )
    implicit_threads = response_g4.choices[0].message.content
    print("--- IMPLICIT THREADS AND DYNAMICS ---")
    print(implicit_threads)
except Exception as e:
    print(f"An error occurred: {e}")

--- IMPLICIT THREADS AND DYNAMICS ---
### Hesitancy or reluctance (even if people “agreed”)
- **The “tentative decision” framing is a tell.** Saying *tentative decision: Tom will work over the weekend* suggests people weren’t fully comfortable owning that choice out loud. It reads like an agreement made under pressure rather than enthusiastic consent.
- **Implicit reluctance around asking Tom to sacrifice personal time.** Nobody is quoted saying “Tom volunteered” or “Tom offered,” and the rationale is to “recover the schedule” (i.e., protect the plan), which can indicate quiet discomfort: it solves the timeline problem by shifting burden onto one person.
- **The concern about compressing testing suggests someone wasn’t buying the optimism.** Even if the team agreed to the weekend work plan, the explicit mention of reduced testing buffer implies at least one person was uneasy about risk and quality—and may have felt they needed to “flag” it to protect themselves later if issues arise.



2.  **Cell 7: g5 - Perform "Intellectual Combinations"**

      * Here, we prompt the AI to be creative and solve a problem by synthesizing different ideas from the meeting.

In [ ]:
# Cell 7: g5 - Generating a Novel Solution
prompt_g5 = f"""
Context: In the meeting, Maria suggested a 'soft launch' to avoid server strain, and also mentioned her team has 'extra bandwidth'.
Tom is facing a 3-day delay on the backend.

Task: Propose a novel, actionable idea that uses Maria's team's extra bandwidth to help mitigate Tom's 3-day delay. Combine these two separate pieces of information into a single solution.
"""
try:
    response_g5 = client.chat.completions.create(
        model="gpt-5.2",
        messages=[{"role": "user", "content": prompt_g5}]
    )
    novel_solution = response_g5.choices[0].message.content
    print("--- NOVEL SOLUTION PROPOSED BY AI ---")
    print(novel_solution)
except Exception as e:
    print(f"An error occurred: {e}")

--- NOVEL SOLUTION PROPOSED BY AI ---
Use Maria’s team’s “extra bandwidth” to run a **soft-launch-focused “backend acceleration squad”** for 72 hours that both reduces immediate load and eliminates Tom’s backend blockers.

**Actionable plan (single combined solution):**
1. **Stand up a Soft Launch Gate + Throttle Layer (owned by Maria’s team)**
   - Implement a feature flag / allowlist (internal users → small pilot cohort) plus request throttling and aggressive caching for the most-used endpoints.
   - Result: you can release on schedule to a limited audience without stressing the backend, buying safe time while the full backend work finishes.

2. **Parallelize Tom’s delayed backend work by offloading “delay-causing” tasks to Maria’s team**
   - Maria’s team takes on the surrounding work that often causes backend delays:
     - Writing/validating API contracts (OpenAPI/Swagger) and generating stubs/mocks.
     - Building a realistic **mock backend** (or contract test harness) so fronte

-----

### Layer 3: Determining the Action (The 'What Next')

Finally, we turn the analysis into concrete, forward-looking artifacts.

1.  **Cell 8: g6 - Define the Output Format (Final Summary)**

      * We compile all the key information into a structured, final summary, showing the importance of clear outputs.

In [ ]:
# Cell 8: g6 - Creating the Final, Structured Summary
prompt_g6 = f"""
Task: Create a final, concise summary of the meeting in a markdown table.
Use the following information to construct the table.

- New Developments: {new_developments}

The table should have three columns: "Topic", "Decision/Outcome", and "Owner".
"""
try:
    response_g6 = client.chat.completions.create(
        model="gpt-5.2",
        messages=[{"role": "user", "content": prompt_g6}]
    )
    final_summary_table = response_g6.choices[0].message.content
    print("--- FINAL MEETING SUMMARY TABLE ---")
    print(final_summary_table)
except Exception as e:
    print(f"An error occurred: {e}")

--- FINAL MEETING SUMMARY TABLE ---
| Topic | Decision/Outcome | Owner |
|---|---|---|
| Backend risk (payment gateway integration) | Identified higher-than-expected complexity; potential **~3-day** backend delay (backend otherwise mostly on track). | Backend team / Tom |
| Schedule & testing impact | A **3-day slip** would compress final testing and could threaten launch due to limited buffer. | Project team |
| Mitigation plan | **Tom will work over the weekend** to try to recover time and avoid delaying the timeline. | Tom |
| Front-end status | Front-end is **ahead of schedule**, providing extra bandwidth. | Front-end team |
| Launch-day risk | Large launch-day social media push could overload servers if early bugs exist. | Product/Engineering |
| Launch strategy | **Soft launch (invite-only) for week 1**, then **major social media push** afterward. | Product/Marketing |


2.  **Cell 9: g7 - Generate the Subsequent Task**

      * The last step is to use the analysis to perform a real-world action, closing the loop from insight to action.

In [ ]:
# Cell 9: g7 - Drafting the Follow-Up Action
prompt_g7 = f"""
Task: Based on the following summary table, draft a polite and professional follow-up email to the team (Sarah, Tom, Maria).
The email should clearly state the decisions made and the action items for each person.

Summary Table:
---
{final_summary_table}
---
"""
try:
    response_g7 = client.chat.completions.create(
        model="gpt-5.2",
        messages=[{"role": "user", "content": prompt_g7}]
    )
    follow_up_email = response_g7.choices[0].message.content
    print("--- DRAFT FOLLOW-UP EMAIL ---")
    print(follow_up_email)
except Exception as e:
    print(f"An error occurred: {e}")

--- DRAFT FOLLOW-UP EMAIL ---
Subject: Follow-up: Payment integration risk, mitigation plan, and launch strategy

Hi Sarah, Tom, and Maria,  

Thanks for today’s discussion. I wanted to summarize the key decisions and confirm the action items coming out of it:

**Decisions / Outcomes**
- **Backend payment gateway integration risk:** We identified higher-than-expected complexity, with a potential **~3-day backend delay** (everything else is largely on track).
- **Schedule & testing impact:** A **3-day slip** would compress final testing and could put launch at risk given the limited buffer.
- **Mitigation approach:** We’ll attempt to recover the time rather than move the overall timeline.
- **Launch strategy to reduce risk:** We will do a **soft launch (invite-only) for week 1**, followed by a **major social media push** after that to avoid overwhelming servers while early issues are still being validated.

**Action Items**
- **Tom (Backend)**
  - Work over the weekend to push the payme

This notebook structure doesn't just "get a summary." It takes the user on a journey, showing them *how* to think with the AI as a partner. It perfectly translates the abstract "Scope, Investigation, Action" framework into a repeatable, educational, and powerful process.